# Report 1 - Residential Energy Consumption

Kevin Zhang

Data Science

1/25/2017


In [32]:
import thinkstats2, thinkplot
import pandas

In [35]:
consumption = pandas.read_csv("ce11-clean.csv")

In [36]:
consumption


,Housing Unit Characteristics and Energy Usage Indicators,Total Housing Units (millions),SEC Total (quadrillion BTU),SEC Per Household (million BTU),SEC Per Household Member million BTU),EE Per Square Foot (thousand BTU),EE Total (Billion Dollars),EE Per Household (dollars),EE Per Household Member (dollars),EE per Square Foot (dollars)
0,Total U.S.,113.6,10.183,89.6,34.9,45.5,229.95,"2,024",787,1.03
1,Northeast,20.8,2.235,107.6,42.4,50.7,53.90,"2,595","1,023",1.22
2,Midwest,25.9,2.914,112.4,45.1,49.5,51.34,"1,981",795,0.87
3,South,42.1,3.220,76.5,30.0,41.0,85.75,"2,038",798,1.09
4,West,24.8,1.814,73.0,26.9,42.7,38.96,"1,568",577,0.92
5,Urban,88.1,7.792,88.5,34.7,47.7,170.29,"1,934",758,1.04
6,Rural,25.5,2.391,93.6,35.3,39.6,59.66,"2,336",882,0.99
7,In metropolitan statistical area,94.0,8.482,90.2,34.9,45.6,189.68,"2,018",780,1.02
8,In micropolitan statistical area,12.4,1.080,87.3,34.8,44.5,24.84,"2,007",800,1.02
9,Not in metropolitan or micropolitan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
consumption['Total Housing Units (millions)']

0     113.6
1      20.8
2      25.9
3      42.1
4      24.8
5      88.1
6      25.5
7      94.0
8      12.4
9       NaN
10      7.2
11     38.8
12     35.4
13     14.1
14     19.1
15      6.3
16     78.6
17     71.8
18      6.7
19     28.1
20      9.0
21     19.1
22      6.9
23     76.5
24     67.1
25      3.8
26      5.5
27     37.1
28     11.4
29     24.3
      ...  
38     15.6
39      2.8
40     24.8
41     24.1
42     18.4
43     14.2
44      9.4
45      6.5
46      4.8
47      8.5
48     31.3
49     35.8
50     18.1
51     15.7
52      7.7
53      5.0
54     23.7
55     27.5
56     21.2
57     14.2
58      9.3
59      5.7
60     12.0
61     16.9
62     11.3
63     85.5
64    101.0
65      6.2
66      4.3
67      2.2
Name: Total Housing Units (millions), dtype: float64